In [0]:
%run "../includes/configuration"

In [0]:
race_results = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
display(race_results)

In [0]:
from pyspark.sql.functions import sum, desc, when, count, col

driver_standings = race_results \
    .groupBy('race_year', 'driver_name', 'driver_nationality', 'team') \
    .agg(sum('points').alias('total_points'), count(when(col('position') == 1, True)).alias('wins')) \
    .orderBy(desc('total_points'))

In [0]:
display(driver_standings)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_df = driver_standings \
    .withColumn('rank', rank().over(driver_rank_spec))
display(final_df)


In [0]:
final_df.write.mode('overwrite').parquet(f"{presentation_folder_path}/driver_standings")